# Benutzung des Tools

1. Unter dem Namen der Datei auf Runtime/Laufzeit drücken.
2. Run all auswählen.
3. Auf Eingabenabfrage warten und diese tätigen.

In [1]:
import pandas_datareader.data as reader
import pandas as pd
import numpy as np
import datetime as dt
import math

## Eingabeaufforderung

Hier die angeforderten Eingaben ausfüllen.

- Die Anzahl der enthaltenen Positionen im Portfolio
- Die Position und die monatliche Sparrate dieser Position
- Das Startdatum des Portfolios

Es lassen sich mit diesem Tool nicht nur ETFs und Aktien, sondern auch Rohstoffe und alle Daten, die über Yahoo Finance zu finden sind analysieren.

In [2]:
## Erstellen des Portfolios und Zeitraumes

# Portfolio zusammensetzen
num_assets = int(input("Wieviele Assets sind in Ihrem Musterportfolio? "))
asset_list = []
rates_list = []
i = 0
for i in range(num_assets):
  asset_list.append(input("Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? "))
  rates_list.append(float(input("Wieviel zahlen monatlich in dieses Asset? ")))
portfolio = dict(zip(asset_list, rates_list))

# Zeitraum bestimmen
formation = input("Bitte geben Sie den Beginn des Portfolios ein (Format: 2015,12,7): ").split(",")
formation = dt.datetime(int(formation[0]), int(formation[1]), int(formation[2]))

Wieviele Assets sind in Ihrem Musterportfolio? 3
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? URTH
Wieviel zahlen monatlich in dieses Asset? 1
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? ^GSPC
Wieviel zahlen monatlich in dieses Asset? 1
Welches Asset würden Sie gerne Ihrem Portfolio hinzufügen? ^GDAXI
Wieviel zahlen monatlich in dieses Asset? 1
Bitte geben Sie den Beginn des Portfolios ein (Format: 2015,12,7): 2020,1,1


In [3]:
## Referenzdaten bestimmen

# Gesamte monatliche Einzahlungen bestimmen
monatliche_einz = 0
for i in rates_list:
  monatliche_einz += i

# Referenzdepots zusammensetzen
referenz_list = ['URTH', '^GSPC','^GDAXI']
referenz_einz = [monatliche_einz, monatliche_einz, monatliche_einz] 
referenz_depot = dict(zip(referenz_list, referenz_einz))

In [4]:
## Funktion zur Erstellung von Depots mit gebenen Einzahlungen
## Wird für eigenes und Referenzdepot verwendet
def rendite_berechnen(ticker_list, einzahlungen):
    
    # Kurse der Ticker einholen und monatliche Rendite berechnen
    df = reader.get_data_yahoo(ticker_list,formation,interval='m')['Adj Close']
    mtl_ret = df.pct_change() + 1
    mtl_ret = mtl_ret.reset_index(drop=True)
    
    
    # Depotwert nach angegebener Zeit berechnen
    depot_werte = pd.DataFrame(0,columns=ticker_list, index=[0])
    for i in range(mtl_ret.shape[0]):
        for ticker in ticker_list:
            if not mtl_ret.loc[i,ticker] == 1.000000:
                depot_werte.loc[0,ticker] += einzahlungen[ticker]
            if not math.isnan(mtl_ret.loc[i,ticker]):
                depot_werte[ticker] *= mtl_ret.loc[i, ticker]
                
    return depot_werte

In [5]:
## Berechnen der Depotwerte (Portfolio und Referenzdepot)

portfolio_werte = rendite_berechnen(asset_list, portfolio)
referenz_werte = rendite_berechnen(referenz_list, referenz_depot)

In [6]:
## Zusammenrechnung der Einzelwerte des Porfolios

# Berechnung des Wertes des gesamten Portfolios
gesamt_depotwert = 0
for column in portfolio_werte:
  gesamt_depotwert += round(portfolio_werte[column][0],2)
  print(f"Ihr {column} wäre auf einen Wert von {round(portfolio_werte[column][0],2)}€ gewachsen.")

print(f"Ihr gesamtes Portfolio wäre somit auf einen Wert von {round(gesamt_depotwert,2)}€ gewachsen.")
print("")

Ihr URTH wäre auf einen Wert von 17.15€ gewachsen.
Ihr ^GSPC wäre auf einen Wert von 16.85€ gewachsen.
Ihr ^GDAXI wäre auf einen Wert von 15.8€ gewachsen.
Ihr gesamtes Portfolio wäre somit auf einen Wert von 49.8€ gewachsen.



In [7]:
## Ausgabe der Referenzwerte

print("Zum Vergleich ein paar Indizes:")
for x in referenz_werte:
  print(f"Ein {x} wäre bei gleicher Sparrate einen Wert von {round(referenz_werte[x][0],2)}€ gewachsen.")

Zum Vergleich ein paar Indizes:
Ein URTH wäre bei gleicher Sparrate einen Wert von 51.44€ gewachsen.
Ein ^GSPC wäre bei gleicher Sparrate einen Wert von 50.56€ gewachsen.
Ein ^GDAXI wäre bei gleicher Sparrate einen Wert von 47.41€ gewachsen.
